# Import Stuff

In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install opencv-python numpy seaborn matplotlib scikit-learn ipykernel tqdm pillow
%pip install onnx onnxruntime quanto

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import time
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import onnx
import onnxruntime
import quanto
from tqdm import tqdm

from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data

from datasets import dataset_utils
from matching import matching
from evaluation.metrics import createPR, recallAt100precision, recallAtK
from datasets.load_dataset import GardensPointDataset, SFUDataset, StLuciaDataset

In [4]:
# GardensPointDataset().load()
# SFUDataset().load()
# StLuciaDataset().load()

# Constants

In [5]:
WEIGHTS_FILE = "calc.caffemodel.pt"
ITERATIONS = 100 # for testing average duration

# Preprocess Images

In [6]:
class ConvertToYUVandEqualizeHist:
    def __call__(self, img):
        img_yuv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        img_rgb = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)
        return Image.fromarray(img_rgb)

preprocess = transforms.Compose(
    [
        ConvertToYUVandEqualizeHist(),
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((120, 160), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
    ]
)

In [7]:
class CustomImageDataset(Dataset):
    def __init__(self, name, folder, transform=None):
        
        self.name = os.path.basename(name)
        self.folder = os.path.join(name, folder)
        self.image_paths = dataset_utils.read_images_paths(self.folder, get_abs_path=True)
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index) :
        image_path = self.image_paths[index]
        img = Image.open(image_path)
        if self.transform:
            img = self.transform(img)
        return(img)

In [8]:
dataset_db = CustomImageDataset("images/SFU", "dry", preprocess)
dataset_q = CustomImageDataset("images/SFU", "jan", preprocess)

print("Dataset Length:", len(dataset_db))
dataset_db[0]

Dataset Length: 385


tensor([[[0.3098, 0.4431, 0.6235,  ..., 0.0314, 0.0196, 0.0196],
         [0.1882, 0.4471, 0.7020,  ..., 0.0471, 0.0353, 0.0353],
         [0.1412, 0.4392, 0.6510,  ..., 0.0431, 0.0314, 0.0353],
         ...,
         [0.7882, 0.8157, 0.8314,  ..., 0.1529, 0.1176, 0.0824],
         [0.7961, 0.8118, 0.8353,  ..., 0.1333, 0.0902, 0.0627],
         [0.7843, 0.8000, 0.8196,  ..., 0.0980, 0.0588, 0.0431]]])

In [9]:
batch_size = 64
num_workers = 8
db_dataloader = DataLoader(dataset_db, batch_size=batch_size, shuffle=False, num_workers=num_workers)
q_dataloader = DataLoader(dataset_q, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Model Definition

In [10]:
class CalcModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.input_dim = (1, 120, 160)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=2, padding=4)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(4, 4), stride=1, padding=2)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = nn.Conv2d(128, 4, kernel_size=(3, 3), stride=1, padding=0)
        self.relu3 = nn.ReLU(inplace=False)
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2)
        self.lrn1 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        self.lrn2 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool(x)
        x = self.lrn1(x)

        x = self.relu2(self.conv2(x))
        x = self.pool(x)
        x = self.lrn2(x)

        x = self.relu3(self.conv3(x))
        x = torch.flatten(x, 1)
        return x

In [11]:
class CalcModelCompile(nn.Module):
    def __init__(self):
        super().__init__()

        self.input_dim = (1, 120, 160)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(5, 5), stride=2, padding=4)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(4, 4), stride=1, padding=2)
        self.relu2 = nn.ReLU(inplace=False)
        self.conv3 = nn.Conv2d(128, 4, kernel_size=(3, 3), stride=1, padding=0)
        self.relu3 = nn.ReLU(inplace=False)
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2)
        self.lrn1 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        self.lrn2 = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)

    @torch.compile
    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.pool(x)
        x = self.lrn1(x)

        x = self.relu2(self.conv2(x))
        x = self.pool(x)
        x = self.lrn2(x)

        x = self.relu3(self.conv3(x))
        x = torch.flatten(x, 1)
        return x

### Normal Model

In [12]:
calc = CalcModel()

# Load the model weights
state_dict = torch.load(WEIGHTS_FILE)
my_new_state_dict = {}
my_layers = list(calc.state_dict().keys())
for layer in my_layers:
    my_new_state_dict[layer] = state_dict[layer]
calc.load_state_dict(my_new_state_dict)

print(calc)

CalcModel(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
  (relu1): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (conv3): Conv2d(128, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
)


### ONNX Model

In [13]:
example_input = torch.randn(1, 1, 120, 160)

dynamic_axes = {"input": {0: "batch_size"}, "output": {0: "batch_size"}}

# Export the model
torch.onnx.export(
    calc,  # model
    example_input,  # example input
    "calc_model.onnx",  # output file name
    input_names=["input"],  # input names
    output_names=["output"],  # output names
    dynamic_axes=dynamic_axes,  # dynamic axes
)

# Load the ONNX model
ort_session = onnxruntime.InferenceSession("calc_model.onnx")

/home/irfan/.conda/envs/bruh/lib/python3.11/site-packages/torch/onnx/_internal/jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/irfan/.conda/envs/bruh/lib/python3.11/site-packages/torch/onnx/utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/home/irfan/.conda/envs/bruh/lib/python3.11/site-packages/torch/onnx/utils.py:1208: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/const

### Dynamic Quantized Model (ONNX)

In [14]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = 'calc_model.onnx'
model_quant = 'calc_model_quant_dynamic.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

# Load the dynamic quantized model
ort_session_quant_dynamic = onnxruntime.InferenceSession("calc_model_quant_dynamic.onnx")

  elem_type: 7
  shape {
    dim {
      dim_value: 5
    }
    dim {
      dim_value: 2
    }
  }
}
.
  elem_type: 7
  shape {
    dim {
      dim_value: 5
    }
    dim {
      dim_value: 2
    }
  }
}
.


### Static Quantized Model (ONNX)

In [15]:
# from onnxruntime.tools.symbolic_shape_infer import SymbolicShapeInference

from onnxruntime.quantization.shape_inference import quant_pre_process

quant_pre_process('calc_model.onnx', 'calc_model_quant_static_prep.onnx')

In [16]:
# calib_ds = db_tensor[:100] # first 100 for calibration - reserve for quantization
# val_ds = db_tensor[100:] # last 100 for validation

calib_ds = torch.stack([dataset_db[i] for i in range(100)])
val_ds = torch.stack([dataset_db[i] for i in range(100, len(dataset_db))])

print(calib_ds.shape)
print(val_ds.shape)

torch.Size([100, 1, 120, 160])
torch.Size([285, 1, 120, 160])


In [17]:
from onnxruntime.quantization.calibrate import CalibrationDataReader

class QuantizationDataReader(CalibrationDataReader):
    def __init__(self, torch_ds, batch_size, input_name):
        self.torch_dl = torch.utils.data.DataLoader(torch_ds, batch_size=batch_size, shuffle=False)
        self.input_name = input_name
        self.datasize = len(self.torch_dl)
        self.enum_data = iter(self.torch_dl)

    def to_numpy(self, pt_tensor):
        return pt_tensor.detach().cpu().numpy() if pt_tensor.requires_grad else pt_tensor.cpu().numpy()

    def get_next(self):
        batch = next(self.enum_data, None)
        if batch is not None:

            data = self.to_numpy(batch[0])
            data = np.expand_dims(data, axis=0)  # Add a new dimension to the data
            
            return {self.input_name: data}
        else:
            return None

    def rewind(self):
        self.enum_data = iter(self.torch_dl)

qdr = QuantizationDataReader(calib_ds, batch_size=64, input_name=ort_session.get_inputs()[0].name)

In [18]:
from onnxruntime.quantization import quantize_static

q_static_opts = {"ActivationSymmetric":False,
                 "WeightSymmetric":True}
# if torch.cuda.is_available():
#     q_static_opts = {"ActivationSymmetric":True,
#                   "WeightSymmetric":True}

# q_static_opts = {"ActivationSymmetric":False, "WeightSymmetric":False}

# check layer quantization support

quantized_model = quantize_static(model_input='calc_model_quant_static_prep.onnx',
                                               model_output='calc_model_quant_static.onnx',
                                               calibration_data_reader=qdr,
                                               extra_options=q_static_opts)

# Load the static quantized model
ort_session_quant_static = onnxruntime.InferenceSession('calc_model_quant_static.onnx')

### Quantization (Quanto)

In [19]:
calc_quanto = CalcModel()

# Load the model weights
state_dict = torch.load(WEIGHTS_FILE)
my_new_state_dict = {}
my_layers = list(calc.state_dict().keys())
for layer in my_layers:
    my_new_state_dict[layer] = state_dict[layer]
calc_quanto.load_state_dict(my_new_state_dict)

print(calc_quanto)

quanto.quantize(calc_quanto, weights=quanto.qint8, activations=quanto.qint8) # quantization is in place
print(calc_quanto)

CalcModel(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
  (relu1): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (conv3): Conv2d(128, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
)
CalcModel(
  (conv1): QConv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
  (relu1): ReLU()
  (conv2): QConv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (conv3): QConv2d(128, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
  (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
)


### Torch Compile

In [20]:
calc_compiled = torch.compile(calc, mode='default')

print(calc_compiled)

OptimizedModule(
  (_orig_mod): CalcModel(
    (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(4, 4))
    (relu1): ReLU()
    (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (relu2): ReLU()
    (conv3): Conv2d(128, 4, kernel_size=(3, 3), stride=(1, 1))
    (relu3): ReLU()
    (pool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=0, dilation=1, ceil_mode=False)
    (lrn1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
    (lrn2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  )
)


# Run Models

### Normal Model

In [21]:
calc.eval()

# Pass database tensor through the model

db_features = []

with torch.no_grad():

    for batch in db_dataloader:
        output = calc(batch)
        db_features.append(output)

db_features = torch.cat(db_features, axis=0)

print(db_features.shape)

# Pass query tensor through the model

q_features = []

with torch.no_grad():

    for batch in q_dataloader:
        output = calc(batch)
        q_features.append(output)

q_features = torch.cat(q_features, axis=0)

print(q_features.shape)

torch.Size([385, 936])
torch.Size([385, 936])


### ONNX Model

In [22]:
# Check if model is a valid ONNX model
onnx_model = onnx.load("calc_model.onnx")
onnx.checker.check_model(onnx_model)

In [23]:
# # Get the input name from the model
# input_name = ort_session.get_inputs()[0].name

# # Ensure the inputs are numpy arrays
# db_matrix = np.array(db_matrix)
# q_matrix = np.array(q_matrix)

# ## Database images

# # Create the input dictionary
# ort_db_input = {input_name: db_matrix}

# # Run the model
# ort_db_output = ort_session.run(None, ort_db_input)

# # Convert the output to a numpy array and print its shape
# ort_db_output = np.array(ort_db_output)
# print(ort_db_output.shape)

# ## Query images

# # Create the input dictionary
# ort_q_input = {input_name: q_matrix}

# # Run the model
# ort_q_output = ort_session.run(None, ort_q_input)

# # Convert the output to a numpy array and print its shape
# ort_q_output = np.array(ort_q_output)
# print(ort_q_output.shape)

### USE NUMPY ARRAYS!!!

In [26]:
from torch.utils.data import DataLoader, TensorDataset

# Get the input name from the model
input_name = ort_session.get_inputs()[0].name

## Database images

for db_inputs in db_dataloader:
    # Create the input dictionary
    # ort_db_input = {input_name: db_inputs[0].detach().cpu().numpy()}

    # print(ort_db_input['input'][0])

    # Run the model
    ort_db_output = ort_session.run(None, ort_db_input)

    # Convert the output to a numpy array and print its shape
    ort_db_output = np.array(ort_db_output)
    print(ort_db_output.shape)

## Query images

for q_inputs in q_dataloader:
    # Create the input dictionary
    ort_q_input = {input_name: q_inputs[0].detach().cpu().numpy()}

    # Run the model
    ort_q_output = ort_session.run(None, ort_q_input)

    # Convert the output to a numpy array and print its shape
    ort_q_output = np.array(ort_q_output)
    print(ort_q_output.shape)

[[0.30980393 0.44313726 0.62352943 ... 0.03137255 0.01960784 0.01960784]
 [0.1882353  0.44705883 0.7019608  ... 0.04705882 0.03529412 0.03529412]
 [0.14117648 0.4392157  0.6509804  ... 0.04313726 0.03137255 0.03529412]
 ...
 [0.7882353  0.8156863  0.83137256 ... 0.15294118 0.11764706 0.08235294]
 [0.79607844 0.8117647  0.8352941  ... 0.13333334 0.09019608 0.0627451 ]
 [0.78431374 0.8        0.81960785 ... 0.09803922 0.05882353 0.04313726]]


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid rank for input: input Got: 3 Expected: 4 Please fix either the inputs/outputs or the model.

In [ ]:
ort_db_output = np.squeeze(ort_db_output)
print(ort_db_output.shape)
ort_q_output = np.squeeze(ort_q_output)
print(ort_q_output.shape)

### Dynamic Quantized Model (ONNX)

In [ ]:
# Check if model is a valid ONNX model
onnx_model_quant_dynamic = onnx.load("calc_model_quant_dynamic.onnx")
onnx.checker.check_model(onnx_model_quant_dynamic)

In [ ]:
ort_session_quant_dynamic = onnxruntime.InferenceSession("calc_model_quant_dynamic.onnx")

In [ ]:
# Get the input name from the model
input_name_quant_dynamic = ort_session_quant_dynamic.get_inputs()[0].name

# Ensure the inputs are numpy arrays
db_matrix = np.array(db_matrix)
q_matrix = np.array(q_matrix)

## Database images

# Create the input dictionary
ort_db_input_quant_dynamic = {input_name: db_matrix}

# Run the model
ort_db_output_quant_dynamic = ort_session_quant_dynamic.run(None, ort_db_input_quant_dynamic)

# Convert the output to a numpy array and print its shape
ort_db_output_quant_dynamic = np.array(ort_db_output_quant_dynamic)
print(ort_db_output_quant_dynamic.shape)

## Query images

# Create the input dictionary
ort_q_input_quant_dynamic = {input_name: q_matrix}

# Run the model
ort_q_output_quant_dynamic = ort_session_quant_dynamic.run(None, ort_q_input_quant_dynamic)

# Convert the output to a numpy array and print its shape
ort_q_output_quant_dynamic = np.array(ort_q_output_quant_dynamic)
print(ort_q_output_quant_dynamic.shape)

### Static Quantized Model (ONNX)

In [ ]:
# Check if model is a valid ONNX model
onnx_model_quant_static = onnx.load("calc_model_quant_static.onnx")
onnx.checker.check_model(onnx_model_quant_static)

In [ ]:
ort_session_quant_static = onnxruntime.InferenceSession("calc_model_quant_static.onnx")

In [ ]:
# Get the input name from the model
input_name_quant_static = ort_session_quant_static.get_inputs()[0].name

# Ensure the inputs are numpy arrays
db_matrix = np.array(db_matrix)
q_matrix = np.array(q_matrix)

## Database images

# Create the input dictionary
ort_db_input_quant_static = {input_name: db_matrix}

# Run the model
ort_db_output_quant_static = ort_session_quant_static.run(None, ort_db_input_quant_static)

# Convert the output to a numpy array and print its shape
ort_db_output_quant_static = np.array(ort_db_output_quant_static)
print(ort_db_output_quant_static.shape)

## Query images

# Create the input dictionary
ort_q_input_quant_static = {input_name: q_matrix}

# Run the model
ort_q_output_quant_static = ort_session_quant_static.run(None, ort_q_input_quant_static)

# Convert the output to a numpy array and print its shape
ort_q_output_quant_static = np.array(ort_q_output_quant_static)
print(ort_q_output_quant_static.shape)

### Quantization (Quanto)

In [ ]:
calc_quanto.eval()

# Pass database tensor through the model

db_features = []

with torch.no_grad():

    for batch in db_dataloader:
        output = calc_quanto(batch)
        db_features.append(output)

db_features = torch.cat(db_features, axis=0)

print(db_features.shape)

# Pass query tensor through the model

q_features = []

with torch.no_grad():

    for batch in q_dataloader:
        output = calc_quanto(batch)
        q_features.append(output)

q_features = torch.cat(q_features, axis=0)

print(q_features.shape)

### Torch Compile

In [ ]:
calc_compiled.eval()

# Pass database tensor through the model

db_features = []

with torch.no_grad():

    for batch in db_dataloader:
        output = calc_compiled(batch)
        db_features.append(output)

db_features = torch.cat(db_features, axis=0)

print(db_features.shape)

# Pass query tensor through the model

q_features = []

with torch.no_grad():

    for batch in q_dataloader:
        output = calc_compiled(batch)
        q_features.append(output)

q_features = torch.cat(q_features, axis=0)

print(q_features.shape)

# Average Time

### Normal Model

In [ ]:
times = [] # Initialize a list to store the time for each pass

for _ in tqdm(range(ITERATIONS), desc="Processing database dataset"):
    start_time = time.time()

    # Pass the dataset through the model
    with torch.no_grad():
        db_features = []
        for batch in db_dataloader:
            output = calc(batch)

    end_time = time.time()
    times.append(end_time - start_time)

average_time = sum(times) / len(times) # Calculate the average time

print(f'Average time: {average_time} seconds')

In [ ]:
times = [] # Initialize a list to store the time for each pass

for _ in tqdm(range(ITERATIONS), desc="Processing query dataset"):
    start_time = time.time()

    with torch.no_grad():
        for batch in q_dataloader:
            output = calc(batch)

    end_time = time.time()
    times.append(end_time - start_time)

average_time = sum(times) / len(times) # Calculate the average time

print(f'Average time: {average_time} seconds')

### Torch Compile

In [ ]:
times = [] # Initialize a list to store the time for each pass

for _ in tqdm(range(ITERATIONS), desc="Processing database dataset"):
    start_time = time.time()

    # Pass the dataset through the model
    with torch.no_grad():
        db_features = []
        for batch in db_dataloader:
            output = calc_compiled(batch)

    end_time = time.time()
    times.append(end_time - start_time)

average_time = sum(times) / len(times) # Calculate the average time

print(f'Average time: {average_time} seconds')

In [ ]:
times = [] # Initialize a list to store the time for each pass

for _ in tqdm(range(ITERATIONS), desc="Processing query dataset"):
    start_time = time.time()

    with torch.no_grad():
        for batch in q_dataloader:
            output = calc_compiled(batch)

    end_time = time.time()
    times.append(end_time - start_time)

average_time = sum(times) / len(times) # Calculate the average time

print(f'Average time: {average_time} seconds')